In [4]:
!pip install gurobipy

# MAXIMUM CLIQUE SIZE PROBLEM

In [27]:
import networkx as nx

def find_max_near_clique(graph):
    max_size = 3
    max_clique = set()
    for clique in nx.find_cliques(graph):
        if len(clique) > max_size:
            max_size = len(clique)
            max_clique = clique
    return max_clique

# Example usage:
G = nx.Graph()
G.add_edges_from([(1, 2), (1, 3),  (1, 4), (1, 5), (1, 6),  (1, 7), (1, 8), (2, 6), (2,7), (2,8),(3,4), (3,5),  (3,7), (4,7), (5, 7),  (5, 8), (6,7), ( 6,8),(7,8) ])
print(find_max_near_clique(G))

[1, 7, 8, 2, 6]


# Largest

In [9]:
import numpy as np
import gurobipy as gp

# Adjacency matrix of the graph
A = np.array([[0, 1, 1, 0],
              [1, 0, 1, 1],
              [1, 1, 0, 1],
              [0, 1, 1, 0]])

# Density vector
d = np.array([2, 3, 4, 5])

# Create a new model from imported gurobi library
m = gp.Model()

# Add binary variables for each node in the graph, These variables represent whether a node is included in the subgraph or not.
x = m.addVars(A.shape[0], vtype=gp.GRB.BINARY)

# Add objective function to maximize the density of the sub graph
m.setObjective(gp.quicksum(d[i]*x[i] for i in range(A.shape[0])), gp.GRB.MAXIMIZE)

# Add constraints for the adjacency matrix using nested loop
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        if A[i,j] == 1:
            m.addConstr(x[i] + x[j] <= 1) # this ensures that two adjacent nodes in the graph cannot both be included in the subgraph.

# Optimize the model
m.optimize()

# Print the optimal solution
print("Optimal subgraph: ", [i for i in range(A.shape[0]) if x[i].x == 1])



Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 4 columns and 20 nonzeros
Model fingerprint: 0x9f3ddf72
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 7.0000000
Presolve removed 10 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 7 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+00, best bound 7.000000000000e+00, gap 0.0000%
Optimal subgraph:  [0, 3]


# with connectivity constrain


In [10]:
import numpy as np
import gurobipy as gp

# Adjacency matrix of the graph
A = np.array([[0, 1, 1, 0],
              [1, 0, 1, 1],
              [1, 1, 0, 1],
              [0, 1, 1, 0]])

# Density vector
d = np.array([2, 3, 4, 5])

# Create a new model from imported gurobi library
m = gp.Model()

# Add binary variables for each node in the graph, These variables represent whether a node is included in the subgraph or not.
x = m.addVars(A.shape[0], vtype=gp.GRB.BINARY)

# Add objective function to maximize the density of the sub graph
m.setObjective(gp.quicksum(d[i]*x[i] for i in range(A.shape[0])), gp.GRB.MAXIMIZE)

# Add constraints for the adjacency matrix using nested loop
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        if A[i,j] == 1:
            m.addConstr(x[i] + x[j] <= 1) # this ensures that two adjacent nodes in the graph cannot both be included in the subgraph.


#connectivity constraint is added to ensure that the resulting subgraph is connected
m.addConstr(gp.quicksum(x[i] for i in range(A.shape[0])) >= 1)

# Optimize the model
m.optimize()

# Print the optimal solution
print("Optimal subgraph: ", [i for i in range(A.shape[0]) if x[i].x == 1])


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 4 columns and 24 nonzeros
Model fingerprint: 0x676248a7
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 7.0000000
Presolve removed 11 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 7 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.000000000000e+00, best bound 7.000000000000e+00, gap 0.0000%
Optimal subgraph:  [0, 3]


# For weighted graph

In [15]:
import numpy as np
import gurobipy as gp

# Adjacency matrix of the graph, specified values are the weights of the edges.
A = np.array([[0, 2, 3, 0],
              [4, 0, 2, 5],
              [1, 3, 0, 4],
              [0, 2, 1, 0]])

# Density vector
d = np.array([2, 3, 4, 5])

# Create a new model from imported gurobi library
m = gp.Model()

# Add binary variables for each node in the graph, These variables represent whether a node is included in the subgraph or not.
x = m.addVars(A.shape[0], vtype=gp.GRB.BINARY)

# Add variables for the edge weights
y = m.addVars(A.shape[0], A.shape[1], vtype=gp.GRB.BINARY)

# Add objective function to maximize the density of the sub graph
m.setObjective(gp.quicksum(d[i]*x[i] for i in range(A.shape[0]))+gp.quicksum(A[i,j]*y[i,j] for i in range(A.shape[0]) for j in range(A.shape[1])), gp.GRB.MAXIMIZE)

# Add constraints for the adjacency matrix using nested loop
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        if A[i,j] != 0:
            m.addConstr(y[i,j] <= x[i])
            m.addConstr(y[i,j] <= x[j])
            m.addConstr(y[i,j] >= x[i] + x[j] - 1)

#connectivity constraint is added to ensure that the resulting subgraph is connected
m.addConstr(gp.quicksum(x[i] for i in range(A.shape[0])) >= 1)

# Optimize the model
m.optimize()

# Print the optimal solution
print("Optimal subgraph: ", [i for i in range(A.shape[0]) if x[i].x == 1])


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 31 rows, 20 columns and 74 nonzeros
Model fingerprint: 0x28177527
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 41.0000000

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 41 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.100000000000e+01, best bound 4.100000000000e+01, gap 0.0000%
Optimal subgraph:  [0, 1, 2, 3]


# Inverse 

In [23]:
import numpy as np
import gurobipy as gp

# Adjacency matrix of the graph
A = np.array([[0, 0, 1, 0],
              [0, 0, 1, 1],
              [1, 1, 0, 1],
              [0, 1, 1, 0]])
# Add binary variables for each node in the graph, These variables represent whether a node is included in the subgraph or not.
m = gp.Model()

x = m.addVars(A.shape[0], vtype=gp.GRB.BINARY)

# Add objective function to minimize the number of nodes in the subgraph
m.setObjective(gp.quicksum(x[i] for i in range(A.shape[0])), gp.GRB.MINIMIZE)

# Add constraints for the adjacency matrix using nested loop
for i in range(A.shape[0]):
    for j in range(i+1,A.shape[1]):
        if A[i,j] == 1:
            m.addConstr(x[i] + x[j] >= 1)

# Optimize the model
m.optimize()

# Print the optimal solution
print("Optimal inverse near clique: ", [i for i in range(A.shape[0]) if x[i].x == 0])


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 4 columns and 8 nonzeros
Model fingerprint: 0x2a47e567
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%
Optimal inverse near clique:  [0, 1]


In [12]:
import numpy as np
import gurobipy as gp

# Adjacency matrix of the graph
A = np.array([[0, 1, 1, 0],
              [1, 0, 1, 1],
              [1, 1, 0, 1],
              [0, 1, 1, 0]])

# Create a new model from imported gurobi library
m = gp.Model()

# Add binary variables for each node in the graph, These variables represent whether a node is included in the subgraph or not.
x = m.addVars(A.shape[0], vtype=gp.GRB.BINARY)

# Add objective function to maximize the number of non-near-clique nodes
m.setObjective(gp.quicksum(x[i] for i in range(A.shape[0])), gp.GRB.MAXIMIZE)

# Add constraints for the adjacency matrix using nested loop
for i in range(A.shape[0]):
    for j in range(i+1,A.shape[1]):
        if A[i,j] == 1:
            m.addConstr(x[i] + x[j] <= 1)

# Optimize the model
m.optimize()

# Print the optimal solution
print("Optimal inverse near clique: ", [i for i in range(A.shape[0]) if x[i].x == 1])

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 4 columns and 10 nonzeros
Model fingerprint: 0xc769fbcb
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%
Optimal inverse near clique:  [0, 3]


# MOTIF 

In [24]:
import numpy as np
import gurobipy as gp

# Adjacency matrix of the graph
A = np.array([[0, 1, 1, 0],
              [1, 0, 1, 1],
              [1, 1, 0, 1],
              [0, 1, 1, 0]])

# Create a new model
m = gp.Model()

# Add binary variables for each node in the graph, These variables represent whether a node is included in the subgraph or not.
x = m.addVars(A.shape[0], A.shape[1], vtype=gp.GRB.BINARY)

# Add objective function to maximize the number of edges in the motif
m.setObjective(gp.quicksum(x[i, j] for i in range(A.shape[0]) for j in range(A.shape[1])), gp.GRB.MAXIMIZE)

# Add constraints to form a clique
for i in range(A.shape[0]):
    m.addConstr(gp.quicksum(x[i, j] for j in range(A.shape[1])) == (A.shape[0]-1))
    for j in range(i+1, A.shape[1]):
            m.addConstr(x[i, j] == x[j, i])

# Optimize the model
m.optimize()

# Print the optimal solution
print("Optimal motif: ", [(i, j) for i in range(A.shape[0]) for j in range(A.shape[1]) if x[i, j].x == 1])


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 16 columns and 28 nonzeros
Model fingerprint: 0x44d4b76d
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective 12.0000000
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolved: 4 rows, 10 columns, 16 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: cutoff, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        12.00000   12.0000

# Motif search problem for MG graph using maximized near clique problem

In [25]:

import networkx as nx

def find_max_cliques(G):
    max_cliques = []
    for clique in nx.find_cliques(G):
        if len(clique) == nx.graph_clique_number(G):
            max_cliques.append(clique)
    return max_cliques

# Example usage
G = nx.Graph()
G.add_edges_from([(1, 6), (1, 11),(1,12),(2,5),(2,7), (2,12), (3, 6), (3, 9),(3,12), (4, 9), (4, 10), (4, 11),(5,2),  (5, 11), (5,12),  (6,12),  (7, 9), (7, 12), (8, 9), (8, 10), (8,11)   ])

max_cliques = find_max_cliques(G)
print(max_cliques)

[[12, 1, 6], [12, 2, 5], [12, 2, 7], [12, 3, 6]]


# Motif search problem for MG graph using largest density sub graph problem

In [29]:
pip install pulp

     -------------------------------------- 14.3/14.3 MB 530.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [30]:
from itertools import combinations
from pulp import *

# Define the size of the motif (e.g. 3)
motif_size = 3

# Define the threshold for the density of the subgraphs (e.g. 0.5)
density_threshold = 0.2

# Define the number of nodes in the graph (e.g. 10)
num_nodes = 4

# Create a list of nodes

nodes = [i for i in range(1,num_nodes+1)]

# Create binary variables for each subgraph of the given size
x = LpVariable.dicts('x', list(combinations(nodes, motif_size)), 0, 1, LpInteger)

# Create the LP problem
prob = LpProblem('Motif Search', LpMaximize)

# Add objective function: maximize the number of subgraphs that have high density
prob += lpSum([x[subgraph] for subgraph in x]), 'Objective'

# Add constraints:
# 1. Each subgraph can only be selected once
for subgraph in x:
    prob += x[subgraph] <= 1

# 2. The density of each selected subgraph must be greater than the density threshold
for subgraph in x:
    prob += (2*len(subgraph) / (motif_size*(motif_size-1))) >= density_threshold

# Solve the LP
status = prob.solve()

# Print the solution
if status == LpStatusOptimal:
    print('Solution:')
    for subgraph in x:
        if x[subgraph].varValue == 1:
            print(subgraph)
else:
    print('No solution found')

C:\Users\velag\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Solution:
(1, 2, 3)
(1, 2, 4)
(1, 3, 4)
(2, 3, 4)


# Considering Dataset

In [5]:
#Inverse
import numpy as np
import pandas as pd
import gurobipy as gp

# Read PPI data into a pandas dataframe
data = pd.read_csv("ppi_data.csv")

# Extract the list of proteins from the dataframe
proteins = list(set(data["protein1"]).union(set(data["protein2"])))

# Create a dictionary to map proteins to indices
protein_index = {proteins[i]: i for i in range(len(proteins))}

# Create an adjacency matrix from the PPI data
A = np.zeros((len(proteins), len(proteins)))
for i in range(data.shape[0]):
    protein1 = protein_index[data.iloc[i]["protein1"]]
    protein2 = protein_index[data.iloc[i]["protein2"]]
    A[protein1, protein2] = 1
    A[protein2, protein1] = 1

# Create a new model
m = gp.Model()

# Add binary variables for each protein
x = m.addVars(len(proteins), vtype=gp.GRB.BINARY)

# Add objective function to minimize the number of edges in the subgraph
m.setObjective(gp.quicksum(x[i]*x[j] for i in range(len(proteins)) for j in range(len(prote

                                                                                      
                                                                                      
                                                                                      
#lARGEST
                                                                                      import numpy as np
import pandas as pd
import gurobipy as gp

# Read PPI data into a pandas dataframe
data = pd.read_csv("ppi_data.csv")

# Extract the list of proteins from the dataframe
proteins = list(set(data["protein1"]).union(set(data["protein2"])))

# Create a dictionary to map proteins to indices
protein_index = {proteins[i]: i for i in range(len(proteins))}

# Create an adjacency matrix from the PPI data
A = np.zeros((len(proteins), len(proteins)))
for i in range(data.shape[0]):
    protein1 = protein_index[data.iloc[i]["protein1"]]
    protein2 = protein_index[data.iloc[i]["protein2"]]
    A[protein1, protein2] = 1
    A[protein2, protein1] = 1

# Create a new model
m = gp.Model()

# Add binary variables for each protein
x = m.addVars(len(proteins), vtype=gp.GRB.BINARY)

# Add objective function to maximize density
m.setObjective(gp.quicksum(x[i]*x[j] for i in range(len(proteins)) for j in range(len(proteins)) if A[i,j] == 1), gp.GRB.MAXIMIZE)

# Add constraints for the adjacency matrix
for i in range(len(proteins)):
    for j in range(len(proteins)):
        if A[i,j] == 1:
            m.addConstr(x[i] + x[j] <= 1)

# Optimize the model
m.optimize()

# Print the optimal solution
print("Optimal subgraph: ", [proteins[i] for i in range(len(proteins)) if x[i].x == 1])

                                                                                      

SyntaxError: invalid syntax (2411441480.py, line 36)